In [14]:
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import jaccard_score, multilabel_confusion_matrix, precision_recall_fscore_support, precision_score
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
from skopt import BayesSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.svm import SVC

In [15]:
x = np.load('../Part1.npy')
Y_t = np.load('../train.npy/y_train.npy')
#X_train = (x-np.min(x))/(np.max(x)-np.min(x))
xt = np.load('../Part2.npy')
Y_test = np.load('../test.npy/Y_Test.npy')
#X_test = (xt-np.min(xt))/(np.max(xt)-np.min(xt))
x_add = np.load('../Part3.npy')
y_add = np.load('../additional.npy/Y_additional.npy')
#x_add = (x_add-np.min(x_add))/(np.max(x_add)-np.min(x_add))
Y_train = np.concatenate((Y_t, Y_test, y_add))
X_train = np.concatenate((x, xt, x_add))
#X_train = (X_train-np.min(X_train))/(np.max(X_train)-np.min(X_train))
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

In [16]:
X_test = np.load('X_single.npy')
Y_test = np.load('Y_single.npy')
scaler = preprocessing.StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [17]:
class OvR_SVM(BaseEstimator, ClassifierMixin):
    def __init__(self, tol, reg, kernel, coef0, degree):
        super(OvR_SVM, self).__init__()
        self.tol = tol
        self.reg = reg
        self.coef0 = coef0
        self.kernel = kernel
        self.degree = degree
        self.model = SVC(tol=self.tol, C = self.reg, class_weight='balanced', kernel = self.kernel, coef0=self.coef0, degree = self.degree)
        self.ovr = OneVsRestClassifier(self.model)

    def fit(self, X_train, Y_train):
        #self.chain = ClassifierChain(self.model, order="random", random_state=1)
        return self.ovr.fit(X_train, Y_train)
    
    def score(self, X_test, Y_test):
        Y_pred_chain = self.ovr.predict(X_test)
        precision = precision_score(Y_test, Y_pred_chain>=0.5, average='samples')
        return precision 

In [18]:
search_space = {
    'tol': (0.0001,0.1),
    'reg': (0.1, 1),
    'kernel': ['poly', 'rbf', 'linear', 'sigmoid','precomputed'],
    'coef0':(0.0,1),
    'degree':(2,8)
}

test = OvR_SVM(0.0001, 1, 'rbf', coef0=0.0, degree = 3)
opt = BayesSearchCV(test, search_space, n_iter=50, cv=7, n_jobs=-1)
opt.fit(X_train, Y_train)

BayesSearchCV(cv=7,
              estimator=OvR_SVM(coef0=0.0, degree=3, kernel='rbf', reg=1,
                                tol=0.0001),
              n_jobs=-1,
              search_spaces={'coef0': (0.0, 1), 'degree': (2, 8),
                             'kernel': ['poly', 'rbf', 'linear', 'sigmoid',
                                        'precomputed'],
                             'reg': (0.1, 1), 'tol': (0.0001, 0.1)})

In [19]:
print("Best parameters found: ", opt.best_params_)
print("Best score: ", opt.best_score_)

Best parameters found:  OrderedDict([('coef0', 0.7483169045407795), ('degree', 7), ('kernel', 'rbf'), ('reg', 0.18622611087047566), ('tol', 0.04680963726079866)])
Best score:  0.24580500077308234


In [20]:
svm = SVC(C=0.18, tol=0.046, kernel='rbf', coef0=0.79, degree=6)
ovr = OneVsRestClassifier(svm)
ovr.fit(X_train, Y_train)

OneVsRestClassifier(estimator=SVC(C=0.18, coef0=0.79, degree=6, tol=0.046))

In [22]:
pred = ovr.predict(X_test)

In [23]:
m = multilabel_confusion_matrix(Y_test, pred)
print(m)

[[[6317    0]
  [ 388    0]]

 [[6200    0]
  [ 505    0]]

 [[6254    0]
  [ 451    0]]

 [[6068    0]
  [ 637    0]]

 [[5945    0]
  [ 760    0]]

 [[6023    0]
  [ 682    0]]

 [[5867  117]
  [ 689   32]]

 [[6079    0]
  [ 626    0]]

 [[6128    0]
  [ 577    0]]

 [[6125    0]
  [ 580    0]]

 [[5927    0]
  [ 776    2]]]
